In [9]:
import cv2
import numpy as np
import mediapipe as mp
import collections

## MediaPipe

In [2]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

In [4]:
#левый угол рта — 61, правый угол — 291
# верхняя середина губ — 13, нижняя середина — 14

LEFT_MOUTH = 61
RIGHT_MOUTH = 291
UPPER_LIP_CENTER = 13
LOWER_LIP_CENTER = 14

In [6]:
def euclid(a, b): return np.linalg.norm(a - b)

In [7]:
def compute_smile_score(landmarks, w, h):
    def p(i):
        lm = landmarks[i]
        return np.array([lm.x * w, lm.y * h], dtype=np.float32)
    left, right = p(LEFT_MOUTH), p(RIGHT_MOUTH)
    up, low = p(UPPER_LIP_CENTER), p(LOWER_LIP_CENTER)
    mid = (up + low) / 2.0

    width = euclid(left, right) + 1e-6
    height = euclid(up, low) + 1e-6
    ratio = width / height
    lift = ((mid[1] - left[1]) + (mid[1] - right[1])) / 2.0
    return ratio, lift, {"left":left, "right":right, "up":up, "low":low, "mid":mid}

def run_video(source=0, ratio_th=2.15, lift_th=2.0, smooth_window=7):
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError("Не удалось открыть видео/камеру")

    with mp_face_mesh.FaceMesh(
        static_image_mode=False, max_num_faces=1, refine_landmarks=False,
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as mesh:
        q_ratio = collections.deque(maxlen=smooth_window)
        q_lift = collections.deque(maxlen=smooth_window)

        while True:
            ok, frame = cap.read()
            if not ok: break

            h, w = frame.shape[:2]
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            res = mesh.process(rgb)

            label = "No face"
            if res.multi_face_landmarks:
                face = res.multi_face_landmarks[0].landmark
                r, l, pts = compute_smile_score(face, w, h)
                q_ratio.append(r)
                q_lift.append(l)

                r_s = float(np.mean(q_ratio))
                l_s = float(np.mean(q_lift))
                smiling = (r_s > ratio_th) and (l_s > lift_th)
                label = f"{'SMILE' if smiling else 'NEUTRAL'} | r={r_s:.2f}, lift={l_s:.1f}px"

                # Рисуем опорные точки
                for key in ["left", "right", "up", "low", "mid"]:
                    x, y = pts[key].astype(int)
                    cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

            cv2.putText(frame, label, (15, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 220, 0), 2)
            cv2.imshow("Smile detection (MediaPipe)", frame)
            if cv2.waitKey(1) & 0xFF == 27:  # ESC
                break

    cap.release()
    cv2.destroyAllWindows()

In [10]:
run_video("data/video_1.mp4")

In [11]:
# FaceMesh индексы
LEFT_MOUTH, RIGHT_MOUTH = 61, 291
UPPER_LIP_CENTER, LOWER_LIP_CENTER = 13, 14
LEFT_EYE_OUT, RIGHT_EYE_OUT = 33, 263  # для нормализации

def euclid(a, b): 
    return np.linalg.norm(a - b)

def compute_features(landmarks, w, h):
    """Возвращает признаки и опорные точки в пикселях + нормализацию на межзрачковое расстояние."""
    def p(i):
        lm = landmarks[i]
        return np.array([lm.x * w, lm.y * h], dtype=np.float32)

    L, R = p(LEFT_MOUTH), p(RIGHT_MOUTH)
    U, D = p(UPPER_LIP_CENTER), p(LOWER_LIP_CENTER)
    M = (U + D) / 2.0
    EL, ER = p(LEFT_EYE_OUT), p(RIGHT_EYE_OUT)

    eye_dist = euclid(EL, ER) + 1e-6        # масштаб лица
    mw = euclid(L, R)                        # ширина рта, px
    mh = euclid(U, D) + 1e-6                 # высота рта, px
    ratio = mw / mh                          # «шире/выше» (безразмерный)
    lift_px = ((M[1] - L[1]) + (M[1] - R[1])) / 2.0   # >0 — уголки выше центра (ось Y вниз)
    lift_n = lift_px / eye_dist              # нормализованный подъём/опускание уголков
    mh_n = mh / eye_dist                     # высота рта в долях расстояния между глазами

    return {
        "ratio": ratio,      # ширина/высота рта
        "lift_n": lift_n,    # подъём уголков (норм.)
        "mh_n": mh_n,        # высота рта (норм.)
    }, {"L":L, "R":R, "U":U, "D":D, "M":M}

def classify_expression(ratio, lift_n,
                        ratio_smile_th=2.20,
                        lift_up_th=0.030,
                        lift_down_th=0.025):
    """
    Пороги:
    - lift_* в долях межзрачкового расстояния (~0.03 ≈ 3% от eye_dist).
    - ratio_smile_th: чем выше, тем «строже» определение улыбки.
    """
    if (ratio > ratio_smile_th) and (lift_n > lift_up_th):
        return "SMILE"
    elif (lift_n < -lift_down_th):
        return "SAD"
    else:
        return "NEUTRAL"

def run_video_3states(source=0, smooth_window=7,
                      ratio_smile_th=2.20, lift_up_th=0.030, lift_down_th=0.025,
                      draw_points=True):
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError("Не удалось открыть видео/камеру")

    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as mesh:
        q_ratio = collections.deque(maxlen=smooth_window)
        q_lift  = collections.deque(maxlen=smooth_window)

        while True:
            ok, frame = cap.read()
            if not ok:
                break

            h, w = frame.shape[:2]
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            res = mesh.process(rgb)

            label = "No face"
            if res.multi_face_landmarks:
                face = res.multi_face_landmarks[0].landmark
                feats, pts = compute_features(face, w, h)

                q_ratio.append(feats["ratio"])
                q_lift.append(feats["lift_n"])

                ratio_s = float(np.mean(q_ratio))
                lift_s  = float(np.mean(q_lift))

                state = classify_expression(
                    ratio_s, lift_s,
                    ratio_smile_th=ratio_smile_th,
                    lift_up_th=lift_up_th,
                    lift_down_th=lift_down_th
                )
                label = f"{state} | r={ratio_s:.2f} lift={lift_s:+.3f}"

                if draw_points:
                    for key in ["L","R","U","D","M"]:
                        x, y = pts[key].astype(int)
                        cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

                # Цвет рамки/текста по классу
                color = {"SMILE": (0,200,0), "SAD": (0,0,220), "NEUTRAL": (200,200,0)}[state]
                cv2.rectangle(frame, (10,10), (w-10, 60), (0,0,0), -1)
                cv2.putText(frame, label, (15, 45), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

            cv2.imshow("Smile/Neutral/Sad (MediaPipe)", frame)
            if cv2.waitKey(1) & 0xFF == 27:  # ESC
                break

    cap.release()
    cv2.destroyAllWindows()

In [24]:
run_video_3states("data/video_1.mp4")

In [25]:
run_video_3states("data/video_2.mp4")

In [26]:
run_video_3states(0)

### 2 часть

In [22]:
import math

In [23]:
# --- Вспомогательные функции ---

def get_distance(p1, p2):
    """Рассчитывает евклидово расстояние между двумя точками."""
    return math.sqrt(((p1[0] - p2[0])**2) + ((p1[1] - p2[1])**2))

def get_ear(lm_list, eye_indices, image_w, image_h):
    """
    Рассчитывает коэффициент пропорциональности глаза (EAR) по 6 точкам.
    
    lm_list: Список всех 468 точек лица
    eye_indices: Индексы 6 точек для одного глаза
    """
    # 
    
    # Преобразуем нормализованные координаты в пиксели
    p1 = (lm_list[eye_indices[0]].x * image_w, lm_list[eye_indices[0]].y * image_h)
    p2 = (lm_list[eye_indices[1]].x * image_w, lm_list[eye_indices[1]].y * image_h)
    p3 = (lm_list[eye_indices[2]].x * image_w, lm_list[eye_indices[2]].y * image_h)
    p4 = (lm_list[eye_indices[3]].x * image_w, lm_list[eye_indices[3]].y * image_h)
    p5 = (lm_list[eye_indices[4]].x * image_w, lm_list[eye_indices[4]].y * image_h)
    p6 = (lm_list[eye_indices[5]].x * image_w, lm_list[eye_indices[5]].y * image_h)

    # Рассчитываем вертикальные расстояния
    vert_dist1 = get_distance(p2, p6)
    vert_dist2 = get_distance(p3, p5)

    # Рассчитываем горизонтальное расстояние
    horiz_dist = get_distance(p1, p4)
    
    if horiz_dist == 0:
        return 0.0

    # Рассчитываем EAR
    ear = (vert_dist1 + vert_dist2) / (2.0 * horiz_dist)
    return ear

# --- Инициализация ---

# Индексы MediaPipe для 6 точек вокруг каждого глаза (P1-P6)
# P1, P4 - горизонтальные края
# P2, P3 - верхнее веко
# P5, P6 - нижнее веко

# ПРАВЫЙ глаз (с точки зрения человека)
RIGHT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
# ЛЕВЫЙ глаз (с точки зрения человека)
LEFT_EYE_INDICES = [362, 385, 387, 263, 373, 380]

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Порог EAR. Если EAR ниже этого значения, считаем глаз закрытым.
# Возможно, вам придется его немного подстроить!
EYE_EAR_THRESHOLD = 0.2

cap = cv2.VideoCapture(0)

# --- Основной цикл ---

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue

    # Получаем размеры кадра
    image_h, image_w, _ = image.shape
    
    # Конвертируем BGR в RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Обработка MediaPipe
    image_rgb.flags.writeable = False
    results = face_mesh.process(image_rgb)
    image_rgb.flags.writeable = True
    
    # Обратно в BGR для OpenCV
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    status = "Eyes Open"
    
    if results.multi_face_landmarks:
        # Получаем точки для первого (и единственного) лица
        face_landmarks = results.multi_face_landmarks[0]
        lm = face_landmarks.landmark
        
        # Рассчитываем EAR для правого глаза
        right_ear = get_ear(lm, RIGHT_EYE_INDICES, image_w, image_h)
        
        # Рассчитываем EAR для левого глаза
        left_ear = get_ear(lm, LEFT_EYE_INDICES, image_w, image_h)
        
        # Если ОБА глаза закрыты (ниже порога)
        if right_ear < EYE_EAR_THRESHOLD and left_ear < EYE_EAR_THRESHOLD:
            status = "Eyes Closed"

    # --- Отображение ---
    
    # Переворачиваем кадр для "зеркального" отображения (как в зеркале)
    flipped_image = cv2.flip(image_bgr, 1)

    # Устанавливаем цвет текста в зависимости от статуса
    color = (0, 0, 255) if status == "Eyes Closed" else (0, 255, 0)
    
    # Рисуем статус на перевернутом кадре
    cv2.putText(flipped_image, status, 
                (30, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, color, 2, cv2.LINE_AA)

    # Показываем результат
    cv2.imshow('MediaPipe Eye Detector', flipped_image)

    # Выход по нажатию 'ESC'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Освобождаем ресурсы
cap.release()
cv2.destroyAllWindows()
face_mesh.close()

### 3 Тело и подъем рук с хлопком над головой счетчик

In [36]:
mp_pose = mp.solutions.pose
PL = mp_pose.PoseLandmark

# Вспомогательные функции
def to_np(lm, w, h):
    return np.array([lm.x * w, lm.y * h], dtype=np.float32)

def euclid(a, b):
    return float(np.linalg.norm(a - b))

def run_pose_clap_counter(
    source=0,
    smooth_window=5,          # сглаживание расстояния между кистями
    clap_thresh_rel=0.35,     # порог «близко»: доля ширины плеч
    min_visibility=0.5,       # минимальная видимость точки
    min_frames_between=6,     # дебаунс: минимум кадров между хлопками
    draw_skeleton=True
):
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError("Не удалось открыть видео/камеру")

    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        smooth_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as pose:

        clap_count = 0
        was_apart = True          # предыдущая фаза (кисти были далеко)
        frames_since_last = 999   # дебаунс-счётчик
        q_dist = collections.deque(maxlen=smooth_window)

        while True:
            ok, frame = cap.read()
            if not ok:
                break
            h, w = frame.shape[:2]
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            res = pose.process(rgb)

            label = "No body"
            arms_up = False

            if res.pose_landmarks:
                lms = res.pose_landmarks.landmark
                # Достаём нужные точки
                pts = {}
                for name in [
                    PL.LEFT_WRIST, PL.RIGHT_WRIST,
                    PL.LEFT_SHOULDER, PL.RIGHT_SHOULDER,
                    PL.LEFT_EYE, PL.RIGHT_EYE
                ]:
                    lm = lms[name.value]
                    pts[name] = (to_np(lm, w, h), lm.visibility)

                # Проверяем видимость ключевых точек
                needed = [PL.LEFT_WRIST, PL.RIGHT_WRIST, PL.LEFT_SHOULDER, PL.RIGHT_SHOULDER, PL.LEFT_EYE, PL.RIGHT_EYE]
                if all(pts[p][1] >= min_visibility for p in needed):
                    LW, RW = pts[PL.LEFT_WRIST][0], pts[PL.RIGHT_WRIST][0]
                    LS, RS = pts[PL.LEFT_SHOULDER][0], pts[PL.RIGHT_SHOULDER][0]
                    LE, RE = pts[PL.LEFT_EYE][0], pts[PL.RIGHT_EYE][0]

                    # Геометрия (в пикселях)
                    shoulder_width = euclid(LS, RS) + 1e-6
                    wrist_dist = euclid(LW, RW)

                    # Нормированное сближение кистей
                    wrist_dist_rel = wrist_dist / shoulder_width
                    q_dist.append(wrist_dist_rel)
                    wrist_dist_smooth = float(np.mean(q_dist))

                    # Руки подняты: обе кисти выше уровня глаз
                    eye_y = min(LE[1], RE[1])
                    arms_up = (LW[1] < eye_y) and (RW[1] < eye_y)

                    # Хлопок: кисти близко И руки подняты
                    close_enough = wrist_dist_smooth < clap_thresh_rel
                    frames_since_last += 1

                    if arms_up and close_enough and was_apart and frames_since_last >= min_frames_between:
                        clap_count += 1
                        was_apart = False
                        frames_since_last = 0
                    elif not close_enough:
                        was_apart = True

                    # Текст состояния
                    state = []
                    state.append("ARMS_UP" if arms_up else "ARMS_DOWN")
                    state.append("CLOSE" if close_enough else "APART")
                    label = f"{' | '.join(state)}  dist={wrist_dist_smooth:.2f}  cnt={clap_count}"

                    # Рисуем направляющие
                    if draw_skeleton:
                        mp.solutions.drawing_utils.draw_landmarks(
                            frame, res.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            landmark_drawing_spec=mp.solutions.drawing_styles.get_default_pose_landmarks_style()
                        )
                        # Линия между запястьями
                        cv2.line(frame, tuple(LW.astype(int)), tuple(RW.astype(int)), (0, 255, 255), 2)
                        # Уровень глаз
                        cv2.line(frame, (0, int(eye_y)), (w, int(eye_y)), (128, 128, 128), 1, cv2.LINE_AA)

            # HUD
            (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
            cv2.rectangle(frame, (10, 10), (20 + tw, 20 + th), (0, 0, 0), -1)
            cv2.putText(frame, label, (15, 15 + th), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 200), 2)
            cv2.rectangle(frame, (w - 160, 10), (w - 10, 70), (0, 0, 0), -1)
            cv2.putText(frame, f"CLAPS: {clap_count}", (w - 150, 55), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 220, 255), 2)

            cv2.imshow("Pose Clap Counter", frame)
            if cv2.waitKey(1) & 0xFF == 27:  # ESC
                break

    cap.release()
    cv2.destroyAllWindows()

In [38]:
run_pose_clap_counter(source=0)

In [44]:
mp_pose = mp.solutions.pose
PL = mp_pose.PoseLandmark

def to_np(lm, w, h): return np.array([lm.x * w, lm.y * h], dtype=np.float32)
def euclid(a, b): return float(np.linalg.norm(a - b))

def run_pose_clap_counter_jump(
    source=0,
    # --- хлопок (как раньше, гистерезис) ---
    smooth_window=5,
    close_th_rel=0.38,
    open_th_rel=0.60,
    min_visibility=0.5,
    min_frames_between=6,
    require_above_head=True,
    use_index_tips=True,
    # --- прыжок ---
    up_start_delta_rel=0.10,      # насколько таз поднялся от "земли", чтобы считать прыжок начатым (доля ширины плеч)
    up_vel_th_rel=0.015,          # минимальная "скорость" вверх (на кадр, в долях ширины плеч)
    ground_return_delta_rel=0.03, # считаем, что вернулись на землю, когда подъём < этого порога
    one_clap_per_jump=True,       # только 1 хлопок за прыжок
    draw_skeleton=True
):
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError("Не удалось открыть видео/камеру")

    with mp_pose.Pose(
        static_image_mode=False, model_complexity=1,
        enable_segmentation=False, smooth_landmarks=True,
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:

        clap_count = 0
        frames_since_last = 999

        # сглаживание расстояния между руками
        q_dist = collections.deque(maxlen=smooth_window)
        prev_smooth = None
        state_close = False

        # прыжок: отслеживаем вертикаль таза
        prev_hip_y = None
        hip_ground_ema = None       # "уровень земли" (эксп. среднее)
        ema_alpha = 0.02            # скорость обновления уровня земли

        jump_state = "GROUND"       # GROUND | ASCEND | AIRBORNE | DESCEND
        clap_in_this_jump = False

        while True:
            ok, frame = cap.read()
            if not ok: break
            h, w = frame.shape[:2]
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            res = pose.process(rgb)

            hud = "No body"
            if res.pose_landmarks:
                lms = res.pose_landmarks.landmark

                names = [
                    PL.LEFT_SHOULDER, PL.RIGHT_SHOULDER,
                    PL.LEFT_EYE, PL.RIGHT_EYE,
                    PL.LEFT_HIP, PL.RIGHT_HIP,
                    PL.LEFT_WRIST, PL.RIGHT_WRIST,
                    PL.LEFT_INDEX, PL.RIGHT_INDEX
                ]
                pts = {n: (to_np(lms[n.value], w, h), lms[n.value].visibility) for n in names}

                need = [PL.LEFT_SHOULDER, PL.RIGHT_SHOULDER, PL.LEFT_EYE, PL.RIGHT_EYE, PL.LEFT_HIP, PL.RIGHT_HIP]
                if all(pts[n][1] >= min_visibility for n in need):

                    # геометрия
                    LS, RS = pts[PL.LEFT_SHOULDER][0], pts[PL.RIGHT_SHOULDER][0]
                    shoulder_width = euclid(LS, RS) + 1e-6
                    mid_shoulders_x = (LS[0] + RS[0]) / 2.0

                    hipL, hipR = pts[PL.LEFT_HIP][0], pts[PL.RIGHT_HIP][0]
                    hip_y = float((hipL[1] + hipR[1]) / 2.0)   # ниже = больше; вверх = меньше

                    # уровень "земли" (обновляем, пока мы реально на земле)
                    if hip_ground_ema is None:
                        hip_ground_ema = hip_y
                    # критерий "похоже стоим": небольшая вертикальная скорость и руки не над головой
                    # (чтобы в верхней фазе прыжка не портить уровень земли)
                    if prev_hip_y is not None:
                        vel_y = hip_y - prev_hip_y              # <0 вверх, >0 вниз
                    else:
                        vel_y = 0.0
                    prev_hip_y = hip_y

                    # нормализованные величины прыжка
                    # чем больше delta_up_rel, тем выше относительно земли мы поднялись
                    delta_up_rel = (hip_ground_ema - hip_y) / shoulder_width
                    vel_up_rel = (-vel_y) / shoulder_width     # >0 движемся вверх

                    # обновление уровня земли (если явно не прыгаем)
                    if abs(vel_up_rel) < 0.005 and delta_up_rel < 0.02:
                        hip_ground_ema = (1 - ema_alpha) * hip_ground_ema + ema_alpha * hip_y

                    # детектор прыжка (простая машина состояний)
                    if jump_state == "GROUND":
                        if delta_up_rel > up_start_delta_rel or vel_up_rel > up_vel_th_rel:
                            jump_state = "ASCEND"
                            clap_in_this_jump = False
                    elif jump_state == "ASCEND":
                        if vel_up_rel <= 0:          # достигли вершины
                            jump_state = "AIRBORNE"
                    elif jump_state == "AIRBORNE":
                        if vel_up_rel < -0.005:      # уверенно пошли вниз
                            jump_state = "DESCEND"
                    elif jump_state == "DESCEND":
                        if delta_up_rel < ground_return_delta_rel:
                            jump_state = "GROUND"

                    # точки рук: указательные или запястья
                    use_index = use_index_tips and pts[PL.LEFT_INDEX][1] >= min_visibility and pts[PL.RIGHT_INDEX][1] >= min_visibility
                    Lp = pts[PL.LEFT_INDEX][0] if use_index else pts[PL.LEFT_WRIST][0]
                    Rp = pts[PL.RIGHT_INDEX][0] if use_index else pts[PL.RIGHT_WRIST][0]

                    # расстояние между руками (норм.)
                    raw_dist = euclid(Lp, Rp)
                    dist_rel = raw_dist / shoulder_width
                    q_dist.append(dist_rel)
                    smooth = float(np.mean(q_dist))
                    deriv = 0.0 if prev_smooth is None else (smooth - prev_smooth)  # >0 расходятся
                    prev_smooth = smooth

                    # руки подняты и над головой?
                    eye_y = min(pts[PL.LEFT_EYE][0][1], pts[PL.RIGHT_EYE][0][1])
                    arms_up = (Lp[1] < eye_y) and (Rp[1] < eye_y)

                    above_ok = True
                    if require_above_head:
                        margin = 0.25 * shoulder_width
                        head_top_y = eye_y - 0.25 * shoulder_width
                        center_ok = (abs(((Lp[0]+Rp[0])/2.0) - mid_shoulders_x) < 0.6 * shoulder_width)
                        above_ok = (Lp[1] < head_top_y and Rp[1] < head_top_y and center_ok)

                    frames_since_last += 1
                    jump_active = jump_state in ("ASCEND", "AIRBORNE", "DESCEND")

                    # --- гистерезис для хлопка ---
                    if arms_up and above_ok and smooth < close_th_rel and jump_active:
                        state_close = True

                    ready_to_count = (
                        state_close and               # были "очень близко"
                        deriv > 0 and                 # минимум расстояния пройден
                        smooth > open_th_rel and      # разошлись достаточно
                        frames_since_last >= min_frames_between and
                        arms_up and above_ok and
                        jump_active and
                        (not one_clap_per_jump or not clap_in_this_jump)
                    )
                    if ready_to_count:
                        clap_count += 1
                        frames_since_last = 0
                        state_close = False
                        clap_in_this_jump = True

                    # HUD
                    state_txt = []
                    state_txt.append(f"JUMP:{jump_state}")
                    state_txt.append("ARMS_UP" if arms_up else "ARMS_DOWN")
                    state_txt.append("ABOVE_OK" if above_ok else "LOW/ASIDE")
                    state_txt.append("CLOSE" if state_close else "OPEN")
                    hud = f"{' | '.join(state_txt)}  d={smooth:.2f} d'={deriv:+.3f}  up={delta_up_rel:+.3f} v_up={vel_up_rel:+.3f}  cnt={clap_count}"

                    if draw_skeleton:
                        mp.solutions.drawing_utils.draw_landmarks(
                            frame, res.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            landmark_drawing_spec=mp.solutions.drawing_styles.get_default_pose_landmarks_style()
                        )
                        # рука-рука
                        cv2.line(frame, tuple(Lp.astype(int)), tuple(Rp.astype(int)), (0,255,255), 2)
                        # линия глаз
                        cv2.line(frame, (0, int(eye_y)), (w, int(eye_y)), (128,128,128), 1, cv2.LINE_AA)
                        # уровень "земли" для таза (визуально)
                        gy = int(hip_ground_ema)
                        cv2.line(frame, (0, gy), (w, gy), (80,80,80), 1, cv2.LINE_AA)

            # отрисовка HUD
            (tw, th), _ = cv2.getTextSize(hud, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            # cv2.rectangle(frame, (10, 10), (20 + tw, 20 + th), (0, 0, 0), -1)
            # cv2.putText(frame, hud, (15, 15 + th), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 200), 2)
            cv2.rectangle(frame, (w - 160, 10), (w - 10, 70), (0, 0, 0), -1)
            cv2.putText(frame, f"CLAPS: {clap_count}", (w - 150, 55), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 220, 255), 2)

            cv2.namedWindow("Pose Clap Counter (jump)", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("Pose Clap Counter (jump)", 1280, 960)

            cv2.imshow("Pose Clap Counter (jump)", frame)
            if cv2.waitKey(1) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

In [48]:
run_pose_clap_counter_jump(source=0)

## MTCNN

In [2]:
import cv2
import math
import numpy as np
from mtcnn import MTCNN

In [3]:
detector = MTCNN()

In [15]:
def classify_mouth(face):
    box = face["box"]  # [x, y, w, h]
    keypoints = face["keypoints"]

    if "mouth_left" not in keypoints or "mouth_right" not in keypoints:
        return "Unknown"

    (ml_x, ml_y) = keypoints["mouth_left"]
    (mr_x, mr_y) = keypoints["mouth_right"]

    face_width = box[2]
    if face_width <= 0:
        return "Unknown"

    mouth_width = math.hypot(mr_x - ml_x, mr_y - ml_y)
    # Нормализуем по ширине лица (чтобы не зависеть от размера в кадре)
    ratio = mouth_width / float(face_width)

    # Пороговые значения ПРИМЕРНЫЕ, подбираются под себя
    if ratio > 0.5:
        return "Happy"
    elif ratio < 0.35:
        return "Sad"
    else:
        return "Neutral"

In [9]:
def eye_contrast(gray_frame, center, face_width):
    """
    Берём небольшой квадратик вокруг глаза, считаем std (контраст).
    Открытый глаз (белок + зрачок) обычно более контрастный, чем закрытый.
    """
    ex, ey = center
    h, w = gray_frame.shape[:2]

    # Размер окна вокруг глаза ~ 15% ширины лица
    size = int(face_width * 0.15)
    if size < 4:
        return 0.0
    half = size // 2

    x1 = max(ex - half, 0)
    x2 = min(ex + half, w - 1)
    y1 = max(ey - half, 0)
    y2 = min(ey + half, h - 1)

    roi = gray_frame[y1:y2, x1:x2]
    if roi.size == 0:
        return 0.0

    return float(np.std(roi))

def classify_eyes_state(frame, face):
    """
    Глаза открыты или закрыты — по контрасту в области глаз.
    """
    box = face["box"]  # [x, y, w, h]
    keypoints = face["keypoints"]
    face_width = box[2]

    if "left_eye" not in keypoints or "right_eye" not in keypoints:
        return "Unknown"

    (le_x, le_y) = keypoints["left_eye"]
    (re_x, re_y) = keypoints["right_eye"]

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    c_left = eye_contrast(gray, (le_x, le_y), face_width)
    c_right = eye_contrast(gray, (re_x, re_y), face_width)
    avg_contrast = (c_left + c_right) / 2.0

    if avg_contrast > 18:  # можно поиграть значением 15–25
        return "Eyes opened"
    else:
        return "Eyes closed"

In [23]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Для удобства делаем "селфи"-зеркало
    frame = cv2.flip(frame, 1)

    # MTCNN работает в RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    faces = detector.detect_faces(rgb)

    if faces:
        # Берём лицо с максимальной уверенностью
        face = max(faces, key=lambda f: f.get("confidence", 0))

        box = face["box"]  # [x, y, w, h]
        x, y, w, h = box

        # Нарисуем рамку лица
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # 1) Классификация рта
        mouth_state = classify_mouth(face)
        mouth_state_text = f"Mouth: {mouth_state}"

        # 2) Классификация глаз
        eyes_state = classify_eyes_state(frame, face)
        eyes_state_text = f"Eyes: {eyes_state}"

        # Нарисуем ключевые точки (глаза, рот, нос)
        for name, (kx, ky) in face["keypoints"].items():
            cv2.circle(frame, (kx, ky), 3, (255, 0, 0), -1)
            cv2.putText(frame, name, (kx + 3, ky - 3),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

    # Выводим текст на экран
    cv2.putText(frame, mouth_state_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.putText(frame, eyes_state_text, (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow("MTCNN", frame)

    # Выход по 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## OpenCV

In [18]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
smile_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_smile.xml"
)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.3, minNeighbors=5, minSize=(80, 80)
    )

    status_text = "The face was not found"

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Берём нижнюю треть лица как область рта
        mouth_y1 = y + int(h * 2 / 3)
        mouth_y2 = y + h
        mouth_roi_gray = gray[mouth_y1:mouth_y2, x:x + w]

        # Пытаемся найти улыбку каскадом
        smiles = smile_cascade.detectMultiScale(
            mouth_roi_gray,
            scaleFactor=1.4,
            minNeighbors=20,
            minSize=(25, 25)
        )

        if len(smiles) > 0:
            status_text = "Happy"
        else:
            # Если нет явной улыбки, смотрим среднюю яркость нижней части лица
            if mouth_roi_gray.size > 0:
                mean_intensity = np.mean(mouth_roi_gray)
            else:
                mean_intensity = 128

            if mean_intensity < 80:
                status_text = "Sad"
            else:
                status_text = "Neutral"

        break  # берём только первое лицо

    cv2.putText(
        frame,
        status_text,
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        (0, 255, 255),
        2,
    )

    cv2.imshow("OpenCV", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [21]:
import cv2

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
eye_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_eye_tree_eyeglasses.xml"
)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.3, minNeighbors=5, minSize=(80, 80)
    )

    eyes_text = ""

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Берём верхнюю половину лица как область глаз
        eyes_roi_gray = gray[y:y + int(h / 2), x:x + w]
        eyes_roi_color = frame[y:y + int(h / 2), x:x + w]

        eyes = eye_cascade.detectMultiScale(
            eyes_roi_gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(20, 20)
        )

        # Рисуем найденные глаза
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(
                eyes_roi_color, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2
            )

        if len(eyes) >= 2:
            eyes_text = "Eyes: opened"
        else:
            eyes_text = "Eyes: closed"

        break

    cv2.putText(
        frame,
        eyes_text,
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        (0, 255, 255),
        2,
    )

    cv2.imshow("OpenCV", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Сравнение MediaPipe, OpenCV и MTCNN для задач курса

## Задачи

1. Определение по губам: **улыбка / грусть / нейтрально**  
2. Определение: **глаза открыты / закрыты**  
3. Подсчёт **хлопков в прыжке** (над головой, в реальном времени)

---

## Сводная таблица

| Задача / Библиотека | MediaPipe (Face / Pose) | OpenCV (Haar / классика) | MTCNN |
|---------------------|-------------------------|---------------------------|--------|
| **1. Улыбка / грусть / нейтрально по рту** | **Лучше всего**: Face Mesh даёт много точных точек губ, можно построить стабильные геометрические признаки | Много ложных срабатываний, сильно зависит от освещения и ракурса | Есть только 2 уголка рта; можно делать простые правила (ширина рта / угол), но сложно отделять тонкие эмоции |
| **2. Глаза открыты / закрыты** | **Лучше всего**: Face/Eye Mesh даёт много точек вокруг века, легко устойчиво ловить моргания и закрытые глаза | Очень чувствительно к очкам, свету и поворотам головы | По центрам глаз вырезаются патчи и анализируются контраст/границы; работает, но сильно зависит от условий съёмки |
| **3. Хлопки в прыжке (над головой)** | **Однозначный выбор**: Pose даёт скелет (кисти, локти, плечи, бёдра, колени) | Практически непригоден для этой задачи без доп. моделей: классические методы OpenCV не дают скелет и плохо отслеживают руки/прыжки | Вообще не подходит: MTCNN видит только лицо и не даёт никакой информации о руках или теле |

---

## Плюсы и минусы

### MediaPipe

**Плюсы:**
- Есть модули под разные части тела: Pose, Face Mesh, Hands, Holistic.
- Хорошо подходит для:
  - позы/жестов (хлопки, прыжки, осанка),
  - анализа лица по геометрии (улыбка, глаза, мимика).
---

### OpenCV (Haar / классические методы)

**Плюсы:**
- Очень простой старт, минимум зависимостей.
- Подходит для демонстрации классического CV:
  - поиск лица,
  - попытка найти глаза и улыбку каскадами.

**Минусы:**
- Сильно зависит от освещения, поворота головы, очков.
- Для тела/жестов практически бесполезен без сторонних моделей.
- Для эмоций и состояния глаз выдаёт много ошибок, всё строится на грубых эвристиках.

---

### MTCNN

**Плюсы:**
- Хороший детектор лица с 5 ключевыми точками:
  - глаза, нос, уголки рта.
- Стабильнее и точнее, чем Haar-каскады, на разных масштабах и при сложном освещении.
- Удобен как первый шаг в пайплайне **распознавания лиц**.

**Минусы:**
- Работает только с **лицом**, тело/руки не видит вообще.
- Для эмоций и состояния глаз даёт мало информации (только несколько точек).

---

## Краткий вывод

- Для **хлопков в прыжке** — практически без альтернатив:  **MediaPipe Pose**.
- Для **улыбка / грусть / нейтрально** и **глаза открыты / закрыты**:
  - учебные, «простые» решения -> можно показать варианты на **OpenCV** и **MTCNN**;
  - более качественное и устойчивое решение -> **MediaPipe Face Mesh / Holistic**.
